In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[*]") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/08 17:45:00 WARN Utils: Your hostname, kirans-mac.local, resolves to a loopback address: 127.0.0.1; using 172.18.197.149 instead (on interface en0)
25/06/08 17:45:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/08 17:45:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/08 17:45:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Create RDD 

In [52]:
ord = spark.sparkContext.textFile('/Users/kiranchinta/Downloads/RetailDB SalesData/Orders')
ordItems = spark.sparkContext.textFile('/Users/kiranchinta/Downloads/RetailDB SalesData/Order_items')

In [53]:
# order items
# itemid, item_orider_id, item_product_id,item_quantity, item_revenue, item_price

In [54]:
ordItems.take(5)

['1,1,957,1,299.98,299.98',
 '2,2,1073,1,199.99,199.99',
 '3,2,502,5,250.0,50.0',
 '4,2,403,1,129.99,129.99',
 '5,4,897,2,49.98,24.99']

# For each product, find its aggregated revenue. using groupByKey()

In [12]:
ordgroup = ordItems.map(lambda x : (int(x.split(',')[2]),float(x.split(',')[4])))

In [13]:
ordgroup.take(5)

[(957, 299.98), (1073, 199.99), (502, 250.0), (403, 129.99), (897, 49.98)]

In [25]:
for k,v in ordgroup.groupByKey().take(5):
    print(k,(v))
    #print(k,list(v))

502 <pyspark.resultiterable.ResultIterable object at 0x1112e5d90>
1014 <pyspark.resultiterable.ResultIterable object at 0x114557bd0>
926 <pyspark.resultiterable.ResultIterable object at 0x113af28d0>
134 <pyspark.resultiterable.ResultIterable object at 0x1145afed0>
276 <pyspark.resultiterable.ResultIterable object at 0x1104f0b50>


In [26]:
# Output:
    # Key: 502, Values: [1, 3]
    # Key: 1014, Values: [2, 4]
    # Key: 926, Values: [5]

In [30]:
ordgroup.groupByKey().mapValues(sum).take(5)

[(502, 3147800.0),
 (1014, 2888993.939999287),
 (926, 14870.69999999995),
 (134, 20025.0),
 (276, 29398.810000000012)]

### For each product, find its aggregated revenue. using reduceByKey()

In [34]:
from operator import add

ordItems.map(lambda x: (int(x.split(',')[2]), float(x.split(',')[4]))) \
        .reduceByKey(add) \
        .take(5)

[(502, 3147800.0),
 (1014, 2888993.9399996493),
 (926, 14870.699999999972),
 (134, 20025.0),
 (276, 29398.809999999947)]

### aggregateByKey()

### Find the maximum revenue for each Order.

In [36]:
ordItems=spark.sparkContext.parallelize([
(2,"Joseph",200), (2,"Jimmy",250), (2,"Tina",130), (4,"Jimmy",50), (4,"Tina",300),
(4,"Joseph",150), (4,"Ram",200), (7,"Tina",200), (7,"Joseph",300), (7,"Jimmy",80)],2)
#Create a Paired RDD
ordPair = ordItems.map(lambda x : (x[0],(x[1],x[2])))

In [37]:
ordPair.collect()

[(2, ('Joseph', 200)),
 (2, ('Jimmy', 250)),
 (2, ('Tina', 130)),
 (4, ('Jimmy', 50)),
 (4, ('Tina', 300)),
 (4, ('Joseph', 150)),
 (4, ('Ram', 200)),
 (7, ('Tina', 200)),
 (7, ('Joseph', 300)),
 (7, ('Jimmy', 80))]

In [42]:
# initial accumulator
acc = 0

# Seqop - apply function to each parition - # acc:initial accumulator # element: every row key's value 
def seq_op(acc,element):
    if acc>element[1]:
        return acc
    else:
        return element[1]

#Combop - apply function across paritions # p1, p2 are returned values from seq_op function
def comb_op(p1,p2):
    if p1 > p2:
        return p1
    else:
        return p2

ordPair.aggregateByKey(acc,seq_op,comb_op).collect()


[(2, 250), (4, 300), (7, 300)]

### Find the maximum revenue for each Order. Print customer name.

In [43]:
# initial accumulator
acc = ("",0)

# Seqop - apply function to each parition
def seq_op(acc,element):
    if acc[1]>element[1]:
        return acc
    else:
        return element


#Combop - apply function across paritions
def comb_op(p1,p2):
    if p1[1] > p2[1]:
        return p1
    else:
        return p2

ordPair.aggregateByKey(acc,seq_op,comb_op).collect()


[(2, ('Jimmy', 250)), (4, ('Tina', 300)), (7, ('Joseph', 300))]

### Sum up all revenue and number of records for each order.

In [44]:
# initial accumulator
acc = 0

# Seqop - apply function to each parition
def seq_op(acc,element):
    return acc+element[1]

#Combop - apply function across paritions
def comb_op(p1,p2):
    return p1+p2

ordPair.aggregateByKey(acc,seq_op,comb_op).collect()

[(2, 580), (4, 700), (7, 580)]

### countByKey()

In [ ]:
### count the number of order by each order_id

In [61]:

ordItems.map(lambda x: (x.split(',')[1],1)).countByKey()

defaultdict(int,
            {'1': 1,
             '2': 3,
             '4': 4,
             '5': 5,
             '7': 3,
             '8': 4,
             '9': 3,
             '10': 5,
             '11': 5,
             '12': 5,
             '13': 1,
             '14': 3,
             '15': 5,
             '16': 2,
             '17': 5,
             '18': 3,
             '19': 2,
             '20': 4,
             '21': 4,
             '23': 1,
             '24': 5,
             '25': 1,
             '27': 3,
             '28': 5,
             '29': 5,
             '30': 1,
             '31': 1,
             '33': 4,
             '34': 1,
             '35': 1,
             '36': 3,
             '37': 2,
             '38': 2,
             '39': 1,
             '41': 2,
             '42': 3,
             '43': 3,
             '44': 1,
             '45': 4,
             '46': 2,
             '48': 2,
             '49': 3,
             '50': 2,
             '51': 2,
             '52': 2,
